
> **ISO2024 INTRODUCTORY SPATIAL 'OMICS ANALYSIS**
>
>
>- HYBRID : TORONTO & ZOOM
>- 10TH JULY 2024 <br>


>**Module 4 PRE-WORK - Supplementary file 1 <br>
>**Instructor : Shamini Ayyadhury**

---

```
Supplementary script 01 :PROCESSING GENE EXPRESSION DATASET

1. For this module, we will use the human brain cancer dataset, that has been stained with the cell segmentation staining kit.
2. Since we want to focus on segmentation, we will perform the following before the main tutorial to generate the following objects from the gene expression dataset.
    * pre-processing
    * normalization/transformation using pearson residuals

In [ ]:
### import the following libraries
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc

import os


In [ ]:
### directory & filepaths
data_dir = '/home/shamini/data1/data_orig/data/spatial/xenium/10xGenomics/cell_seg_brain_cancer/'
out = '/home/shamini/data/projects/spatial_workshop/out/module4/'
os.makedirs(out, exist_ok=True) # create a new directory to store the output files


In [ ]:
adata = sc.read_10x_h5(data_dir+'out/cell_feature_matrix.h5')
df_cell = pd.read_parquet(data_dir+'out/cells.parquet')
df_cell.set_index('cell_id', inplace=True)
df_cell.reindex(adata.obs.index)
adata.obs = df_cell
### process the adata object
sc.pp.filter_cells(adata, min_counts=9)
sc.pp.filter_genes(adata, min_cells=3)
### reduce adata size to fit the cropped image coordinates
#adata = adata[adata.obs.index.isin(df_cells_subset['cell_id']),:]
adata

In [ ]:
import umap

sc.experimental.pp.normalize_pearson_residuals(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata, n_neighbors=25)


reducer = umap.UMAP()
embedding = reducer.fit_transform(adata.obsm['X_pca'])

adata.obsm['X_umap'] = embedding

adata.uns['spatial'] = adata.obs[['x_centroid', 'y_centroid']]


In [ ]:
adata.write_h5ad(out+'adata.h5ad')